# 第4章　標準付属ツール

PostgreSQLは、データベースの管理・運用に関する様々なツールを提供している。

## 4-1　共通的なオプション

In [ ]:
initdb -?

## 4-2　pg_ctl

PostgreSQLの起動、停止、状態確認などに使う管理ツールである。

### 4-2-1　pg_ctl initdb

データベースクラスタの作成を行う。

In [ ]:
pg_ctl initdb -D /pgdata -o '--encoding=UTF8 --no-locale'

### 4-2-2　pg_ctl start

PostgreSQLをバックグラウンドで起動する。

PostgreSQLの起動を待つ →　pg_ctl start が開始する

待ち時間の指定ができ、それをオーバーすると失敗する。<br>
ただし、失敗するのは pg_ctl start であって、PostgreSQL自体は起動する。

In [ ]:
pg_ctl start -D /pgdata -t 120

### 4-2-3　pg_ctl stop

PostgreSQLを停止（シャットダウン）する。

#### シャットダウンモード

smart：クライアントからの接続がすべて切断されるまで待ってから停止する（スマートシャットダウン）<br>
fast：クライアントからの接続を強制的に切断してから停止する（高速シャットダウン）<br>
immediate：クリーンアップ処理をせず緊急停止する、次回の起動に復旧処理が必要になる（即時シャットダウン）<br>

デフォルト：高速シャットダウン、停止までの待ち時間は60秒

停止時間だけ待つ → 各停止方法の実行　となる。（待たない場合は、-Wを使う）

In [ ]:
pg_ctl stop -D /pgdata -m smart

-m smart でスマートシャットダウンを指定している。

### 4-2-4　pg_ctl restart

PostgreSQLを再起動する。

停止までの時間を指定でき、それをオーバーすると失敗する。<br>
ただし、失敗するのは pg_ctl restart であり、PostgreSQLの停止自体は継続される。

In [ ]:
pg_ctl restart -D /pgdata -m f

-m f で高速シャットダウンを指定している。

### 4-2-5　pg_ctl reload

PostgreSQLに設定ファイルを再読み込みさせる。

In [ ]:
pg_ctl reload -D /pgdata

### 4-2-6　pgctl_status

PostgreSQLが起動しているかどうかを確認する。

In [ ]:
pg_ctl status -D /pgdata

### 4-2-7　pg_ctl kill

プロセスにシグナルを送信する。

マスタプロセスに送信するために、PIDを調べる必要があるので、最初にpg_ctl statusを実行する。

In [ ]:
pg_ctl status -D /pgdata

pg_ctl kill TERM

同じ効果を持つコマンドごとにまとめておく。

In [ ]:
pg_ctl kill TERM

pg_ctl stop -m smart

In [ ]:
pg_ctl kill INT

pg_ctl stop -m fast

In [ ]:
pg_ctl kill QUIT

pg_ctl stop -m immediate

In [ ]:
pg_ctl kill HUP

pg_ctl reload

## 4-3　ツールから PostgreSQL への接続

-h --host 接続先のホスト名：デフォルトはPGHOST、それもなければ「ツールを実行をしたホスト」が接続先となる<br>
-p --port ポート番号：デフォルトはPGPORT、それもなければ「5432」が指定される<br>
-U --username ユーザ名：デフォルトはPGUSER、それもなければ「ツールを実行したOSユーザ名」が使われる

In [ ]:
psql -h 192.188.0.10 -p 9999 -U user1

### 4-3-1　データベースユーザ

PostgreSQLとOSのユーザは全く別物である。

CREATEROLE：ユーザを作成する権限<br>
CREATEDB：データベースを作成する権限<br>
SUPERUSER：データベースに対するあらゆる操作が可能な権限<br>
LOGIN：データベースに接続できる権限<br>
REPLICATION：ストミングレプリケーションとして接続するための権限

権限を必要とする操作を行う場合は、その権限を持ったユーザに接続して実行する必要がある。

また、データベースユーザはデータベースクラスタで共通（一意）である。

## 4-4　createuser

データベースユーザを作成するコマンドである。<br>
下記のオプションで、権限を与えることができる。

-P、--pwprompt：パスワードを設定する<br>
-s、--superuser：スーパーユーザ権限を設定する<br>
-S、--no-superuser：スーパーユーザ権限を設定しない（デフォルト）<br>
-d、--createdb：データベースの作成権限を設定する<br>
-D、--no-createdb：データベースの作成権限を設定しない<br>
-r、--createrole：ユーザの作成権限を設定する<br>
-R、--no-createrole：ユーザの作成権限を設定しない<br>
-l、--login：ログイン権限を設定する<br>
-L、--no-login：ログイン権限を設定しない<br>
--interractive：対話的に設定する場合に指定<br>
-e、--echo：createuserが実行したSQLを出力

大文字と小文字のあるコマンドの場合は、大文字のものがネガティブな意味を持っている。

デフォルトでログイン権限を持っている。<br>
また、スーパーユーザ権限はデータベースとユーザの作成権限も持つ。

データベースクラスタには、PostgreSQLの管理ユーザと同じ名前のデータベースユーザ（PostgeSQLユーザ）が最初から定義されている。<br>また、このユーザはスーパユーザである。

## 4-5　dropuser

データベースユーザを削除するコマンドである。

In [ ]:
createuser -S -D -R -L user2

dropuser -f user2

## 4-6　createdb

データベースを作成するコマンドである。

In [ ]:
createdb -U user1 examdb

エンコーディングとロケールをテンプレートと異なるものに設定する場合には、テンプレート0を選択する必要がある。

In [ ]:
createdb -E EUC_JP -l C -T template testdb

psql -lで、データベースの所有者・エンコーディング・ロケールを確認できる。

In [ ]:
psql -l

## 4-7　dropdb

データベースを削除するコマンドである。

In [ ]:
dropdb -U postgres -i testdb

## 4-8　psql

### 4-8-1　データベースへの接続と切断

psqlは、PostgreSQLへの**接続**・SQLコマンドの発行・結果の表示などを行うコマンドである。
下記のようなオプションが用意されている。

-l、--list:データベースの一覧を表示し、psqlを終了する<br>
-C、--command="コマンド"：指定したコマンドを実行し、表示後にpsqlを終了する<br>
-f、--file=ファイル名：指定したファイルからコマンドを読み込んで実行し、表示後にpsqlを終了する<br>
-d、--dbname=データベース名：接続するデータベースの名前を指定する、d省略でdbnameとなる



In [ ]:
psql -U user1 examdb

PostgreSQLへの接続を切断し、psqlを終了するには、下記のどちらかを使用すれば良い。

In [ ]:
\q

\quit

### 4-8-2　SQLコマンドの入力

SQLコマンドは、psqlがPostgreSQLに送信→PostgreSQLが実行→psqlが受け取って表示する。

また「->」はコマンドの途中であることを表す。

### 4-8-3　メタコマンド

psqlはSQLコマンド以外にも、\から始まるメタコマンドを入力できる。<br>
ただし、あくまで実行するのはpsqlではなくPostgreSQLである。

\q：psqlを終了する<br>
\l、\list：データベースの一覧を表示する（psql -lと同じ効果）<br>
\d パターン：パターン：指定したパターンにマッチしたオブジェクトの名前を表示する<br>
\d：テーブル、ビュー、シーケンスの一覧を表示する<br>
\du：データベースユーザの一覧を表示する<br>
\dn：スキーマの一覧を表示する<br>
\dt：テーブルの一覧を表示する<br>
\di：インデックスの一覧を表示する<br>
\dv：ビューの一覧を表示する<br>
\ds：シーケンスの一覧を表示する<br>
\dS：システムカタログの一覧を表示する<br>
\df：関数の一覧を表示する<br>
\dp：テーブル、ビュー、シーケンスの一覧と、それらに設定されているテーブル単位の情報を表示する<br>
\z：\dpの別名<br>
\copy：PostgreSQLとpsqlの間でテーブルデータをコピーする<br>
\password [ユーザ名]：指定したデータベースユーザ（デフォルトは現在のユーザ）のパスワードを変更する<br>
\c、\connect データベース名：現在のデータベースを切断ｓ、指定したデータベースに新規接続する<br>
\x：結果の表示形式を拡張モード（1→複数行）に変更する、再度実行すると元に戻る<br>
\?：メタコマンドの一覧を表示する<br>
\h、\help SQLコマンド：SQLコマンドのヘルプ情報を表示する、未指定でコマンドの一覧を表示する

メタコマンドは、SQLコマンドと異なり、改行で終わりが判断される。

### 4-8-4　コマンドのバッチ実行

psqlでは、バッチ的（非対話的）にコマンドを実行することができる。<br>
その際は、実行するコマンドをファイルに記述し、そのファイルを-fオプションで指定して、psqlを開始すれば良い。

In [ ]:
cat exam.sql
select current_user, current_date;
\du
psql -f exam.sql -U user1 examdb

psqlでは、-cオプションでもバッチ的に実行できる。<br>
ただし、SQLコマンドとメタコマンドを混在させることはできない。